In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "USDJPY"
MINUTES = 1
EVENT = '58m-dollar'
START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
# RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{EVENT}-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
NORMALIZED_DIR = BASE_DIR / "normalized"
SCALER_DIR = BASE_DIR / "scalers"

# Final paths
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"
NORMALIZED_FILE_PATH = NORMALIZED_DIR / f"{RESAMPLED_NAME}_normalized.pkl"
STD_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_standard_scaler.pkl"
MINMAX_SCALER_PATH = SCALER_DIR / f"{RESAMPLED_NAME}_minmax_scaler.pkl"

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-03 22:47:00,103.2105,103.2165,103.2045,103.2135,3.689000e+10,0.031562,24.331206,0.0000,0.000000,0.000000,...,0.866025,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 22:48:00,103.2155,103.2175,103.2095,103.2160,5.710000e+10,0.028810,24.768070,0.0025,0.000024,0.000024,...,0.866025,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 22:49:00,103.2120,103.2140,103.2105,103.2120,5.791000e+10,0.037950,24.782156,-0.0040,-0.000039,-0.000039,...,0.866025,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 22:50:00,103.2105,103.2135,103.2105,103.2120,2.859000e+10,0.037818,24.076323,0.0000,0.000000,0.000000,...,0.866025,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025
2021-01-03 22:51:00,103.2070,103.2135,103.2045,103.2115,2.205000e+10,0.034875,23.816578,-0.0005,-0.000005,-0.000005,...,0.866025,6,-0.781831,0.62349,3,0.571268,0.820763,1,0.5,0.866025


In [4]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'log_volume',
       'close_delta', 'close_return', 'close_log_return', 'ret_mean_5',
       'ret_mean_10', 'ema5', 'ema5_slope', 'ema20', 'ema20_slope', 'atr14',
       'atr20', 'vol_adj_return', 'close_to_atr', 'bb_upper', 'bb_lower',
       'bb_mavg', 'bb_width', 'bb_position', 'donchian_upper',
       'donchian_lower', 'donchian_mid', 'donchian_width', 'stoch_k',
       'stoch_d', 'rsi14', 'macd', 'macd_signal', 'macd_diff', 'unix_time',
       'hour', 'hour_sin', 'hour_cos', 'dow', 'dow_sin', 'dow_cos', 'dom',
       'dom_sin', 'dom_cos', 'month', 'month_sin', 'month_cos'],
      dtype='object')

In [5]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,...,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06
mean,1.333599e+02,1.333709e+02,1.333489e+02,1.333600e+02,4.621734e+11,7.371343e-03,2.624949e+01,3.617044e-05,2.947861e-07,2.820617e-07,...,-3.159345e-03,2.043148e+00,3.444055e-01,-4.867108e-02,1.570863e+01,2.627637e-03,-2.241271e-02,6.520214e+00,-4.980980e-03,-6.688097e-03
std,1.675104e+01,1.675366e+01,1.674836e+01,1.675104e+01,4.804366e+11,1.091137e-02,1.300103e+00,2.215688e-02,1.595134e-04,1.595330e-04,...,7.070467e-01,1.499731e+00,5.273959e-01,7.751581e-01,8.773026e+00,7.139855e-01,6.997972e-01,3.434585e+00,7.077746e-01,7.063896e-01
min,1.026025e+02,1.026055e+02,1.025925e+02,1.026025e+02,0.000000e+00,1.000000e-03,0.000000e+00,-2.065000e+00,-1.380804e-02,-1.390426e-02,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.145905e+02,1.145965e+02,1.145840e+02,1.145905e+02,1.197400e+11,3.973684e-03,2.550859e+01,-7.500000e-03,-5.791981e-05,-5.792149e-05,...,-7.071068e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-8.660254e-01
50%,1.365375e+02,1.365507e+02,1.365240e+02,1.365375e+02,3.039700e+11,5.715596e-03,2.644019e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.836970e-16,2.000000e+00,4.338837e-01,-2.225209e-01,1.600000e+01,-2.449294e-16,-5.064917e-02,7.000000e+00,-2.449294e-16,-1.836970e-16
75%,1.477040e+02,1.477155e+02,1.476920e+02,1.477040e+02,6.463200e+11,7.305263e-03,2.719456e+01,8.000000e-03,5.968590e-05,5.968412e-05,...,7.071068e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,9.000000e+00,5.000000e-01,5.000000e-01
max,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,5.114110e+12,3.740000e-01,2.926302e+01,1.579500e+00,1.228753e-02,1.221265e-02,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [6]:
FEATURES_COLS = [
    # Basic Data
    'close_log_return',
    'log_volume',
    'spread',

    # Other
    'ret_mean_5',
    'ret_mean_10'
]
TIME_FEATURE_COLS = [
    'hour_cos',
    'dow_cos',
    'dom_cos',
    'month_cos',
]
TA_MIN_MAX = [
    # TA
    'rsi14',
]
TA_STD = [
    "ema5",
    "ema5_slope",
    "ema20",
    "ema20_slope",
    "atr14",
    "atr20",
    "vol_adj_return",
    "close_to_atr",
    "bb_upper",
    "bb_lower",
    "bb_mavg",
    "bb_width",
    "bb_position",
    "donchian_upper",
    "donchian_lower",
    "donchian_mid",
    "donchian_width",
    "stoch_k",
    "stoch_d",
    "macd",
    "macd_signal",
    "macd_diff",
]

In [7]:
COLS_TO_STD = FEATURES_COLS + TA_STD
COLS_TO_MIN_MAX = TA_MIN_MAX

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [9]:
df[COLS_TO_STD] = standard_scaler.fit_transform(df[COLS_TO_STD])
df[COLS_TO_MIN_MAX] = minmax_scaler.fit_transform(df[COLS_TO_MIN_MAX])

In [10]:
df.describe()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
count,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,...,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06,1.487264e+06
mean,1.333599e+02,1.333709e+02,1.333489e+02,1.333600e+02,4.621734e+11,-4.892176e-17,1.174122e-15,3.617044e-05,2.947861e-07,1.351081e-17,...,-3.159345e-03,2.043148e+00,3.444055e-01,-4.867108e-02,1.570863e+01,2.627637e-03,-2.241271e-02,6.520214e+00,-4.980980e-03,-6.688097e-03
std,1.675104e+01,1.675366e+01,1.674836e+01,1.675104e+01,4.804366e+11,1.000000e+00,1.000000e+00,2.215688e-02,1.595134e-04,1.000000e+00,...,7.070467e-01,1.499731e+00,5.273959e-01,7.751581e-01,8.773026e+00,7.139855e-01,6.997972e-01,3.434585e+00,7.077746e-01,7.063896e-01
min,1.026025e+02,1.026055e+02,1.025925e+02,1.026025e+02,0.000000e+00,-5.839181e-01,-2.019033e+01,-2.065000e+00,-1.380804e-02,-8.715783e+01,...,-1.000000e+00,0.000000e+00,-7.818315e-01,-9.009689e-01,1.000000e+00,-9.987165e-01,-9.948693e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,1.145905e+02,1.145965e+02,1.145840e+02,1.145905e+02,1.197400e+11,-3.113872e-01,-5.698804e-01,-7.500000e-03,-5.791981e-05,-3.648372e-01,...,-7.071068e-01,1.000000e+00,0.000000e+00,-9.009689e-01,8.000000e+00,-7.247928e-01,-7.587581e-01,4.000000e+00,-8.660254e-01,-8.660254e-01
50%,1.365375e+02,1.365507e+02,1.365240e+02,1.365375e+02,3.039700e+11,-1.517452e-01,1.466835e-01,0.000000e+00,0.000000e+00,-1.768047e-03,...,-1.836970e-16,2.000000e+00,4.338837e-01,-2.225209e-01,1.600000e+01,-2.449294e-16,-5.064917e-02,7.000000e+00,-2.449294e-16,-1.836970e-16
75%,1.477040e+02,1.477155e+02,1.476920e+02,1.477040e+02,6.463200e+11,-6.056067e-03,7.269192e-01,8.000000e-03,5.968590e-05,3.723498e-01,...,7.071068e-01,3.000000e+00,7.818315e-01,6.234898e-01,2.300000e+01,7.247928e-01,6.889669e-01,9.000000e+00,5.000000e-01,5.000000e-01
max,1.619500e+02,1.619510e+02,1.619435e+02,1.619495e+02,5.114110e+12,3.360063e+01,2.317920e+00,1.579500e+00,1.228753e-02,7.655074e+01,...,1.000000e+00,6.000000e+00,9.749279e-01,1.000000e+00,3.100000e+01,9.987165e-01,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00


In [11]:
print(df[COLS_TO_STD].mean())         # Should be ~0
print(df[COLS_TO_STD].std(ddof=0))

close_log_return    1.351081e-17
log_volume          1.174122e-15
spread             -4.892176e-17
ret_mean_5          6.115220e-18
ret_mean_10        -1.375925e-18
ema5               -6.261986e-16
ema5_slope          3.172271e-18
ema20              -3.130993e-16
ema20_slope         3.745572e-18
atr14              -1.956870e-17
atr20              -2.152558e-16
vol_adj_return     -7.644025e-19
close_to_atr        1.375925e-17
bb_upper           -4.696489e-16
bb_lower            7.827482e-17
bb_mavg            -1.565496e-16
bb_width           -6.849047e-17
bb_position         2.323784e-17
donchian_upper      4.696489e-16
donchian_lower     -3.913741e-16
donchian_mid        7.827482e-17
donchian_width     -5.870611e-17
stoch_k            -1.481412e-16
stoch_d            -2.996458e-17
macd               -2.216767e-18
macd_signal         5.733019e-18
macd_diff          -6.505782e-18
dtype: float64
close_log_return    1.0
log_volume          1.0
spread              1.0
ret_mean_5          1.

In [12]:
import joblib
joblib.dump(standard_scaler, STD_SCALER_PATH)
joblib.dump(minmax_scaler, MINMAX_SCALER_PATH)
df.to_pickle(NORMALIZED_FILE_PATH)